In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import json
import sqlite3


In [13]:
#Creating the user table 
conn = sqlite3.connect('airport.db')
cursor = conn.cursor()

cursor.execute("""
               CREATE TABLE IF NOT EXISTS user(
               email TEXT PRIMARY KEY,
               name TEXT)""")

conn.commit()
conn.close()


In [14]:
#Creating the table for the flight details
conn = sqlite3.connect('airport.db')
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS flights (
    flight_id INTEGER PRIMARY KEY AUTOINCREMENT,
    source TEXT,
    destination TEXT,
    date TEXT,
    time TEXT,
    seats_available INTEGER
) """
)
conn.commit()
conn.close()

In [15]:
#funcation to book the flight

In [3]:
conn = sqlite3.connect("airport.db")
cursor = conn.cursor()

# 1️⃣ Delete existing bookings table
cursor.execute("DROP TABLE IF EXISTS bookings")

# 2️⃣ Create new bookings table
cursor.execute("""
CREATE TABLE bookings (
    booking_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_email TEXT,
    flight_id INTEGER,
    status TEXT,
    FOREIGN KEY(user_email) REFERENCES users(email),
    FOREIGN KEY(flight_id) REFERENCES flights(flight_id)
)
""")

# 3️⃣ Insert sample data
bookings_data = [
    ("ali@gmail.com", 1, "Booked"),
    ("ahmed@gmail.com", 2, "Booked"),
    ("hassan@gmail.com", 3, "Cancelled")
]

cursor.executemany("""
INSERT INTO bookings (user_email, flight_id, status)
VALUES (?, ?, ?)
""", bookings_data)

conn.commit()
conn.close()

print("Bookings table recreated and data inserted successfully.")

Bookings table recreated and data inserted successfully.


In [4]:
#function to view the older tables 

def show_tables():
    conn = sqlite3.connect("airport.db")
    cursor = conn.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    conn.close()

    print("Existing Tables:")
    for table in tables:
        print(table[0])

show_tables()

Existing Tables:
user
sqlite_sequence
flights
bookings


Funcations to insert the data into tables

In [5]:
def insert_users(email,name):
    conn = sqlite3.connect("airport.db")
    cursor = conn.cursor()
    

    cursor.execute(
        "INSERT INTO user (email, name) VALUES (?, ?)",
        (email,name)
    )

    conn.commit()
    conn.close()

    return "Users inserted successfully."

In [6]:
def show_columns(table_name):
    conn = sqlite3.connect("airport.db")
    cursor = conn.cursor()

    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()

    conn.close()

    print(f"\nColumns in '{table_name}' table:")
    for col in columns:
        print(f"Column Name: {col[1]} | Type: {col[2]}")

# Example usage
show_columns("flights")


Columns in 'flights' table:
Column Name: flight_id | Type: INTEGER
Column Name: source | Type: TEXT
Column Name: destination | Type: TEXT
Column Name: date | Type: TEXT
Column Name: time | Type: TEXT
Column Name: seats_available | Type: INTEGER


In [7]:

# function to insert data into flight details
def insert_flights(place):
    conn = sqlite3.connect("airport.db")
    cursor = conn.cursor()

    # Optional: clear old data before inserting
    cursor.execute("DELETE FROM flights")

    cursor.execute(
        "SELECT * FROM flights WHERE values =? ",
        (place,) 
    )

    conn.commit()
    conn.close()

    print("Flights inserted successfully.")



In [8]:
show_columns("user")
show_columns("flights")
show_columns("bookings")


Columns in 'user' table:
Column Name: email | Type: TEXT
Column Name: name | Type: TEXT

Columns in 'flights' table:
Column Name: flight_id | Type: INTEGER
Column Name: source | Type: TEXT
Column Name: destination | Type: TEXT
Column Name: date | Type: TEXT
Column Name: time | Type: TEXT
Column Name: seats_available | Type: INTEGER

Columns in 'bookings' table:
Column Name: booking_id | Type: INTEGER
Column Name: user_email | Type: TEXT
Column Name: flight_id | Type: INTEGER
Column Name: status | Type: TEXT


In [9]:
insert_flights()


TypeError: insert_flights() missing 1 required positional argument: 'place'

In [10]:
def view_users():
    conn = sqlite3.connect("airport.db")
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM user")
    rows = cursor.fetchall()

    conn.close()

    print("\nUsers Table:")
    for row in rows:
        print(row)

In [11]:
insert_users('ikrammir69@gmail.com','ikram')

IntegrityError: UNIQUE constraint failed: user.email

In [12]:
view_users()


Users Table:
('ali@gmail.com', 'Ali')
('ahmed@gmail.com', 'Ahmed')
('hassan@gmail.com', 'Hassan')
('ikrammir69@gmail.com', 'ikram')


In [13]:
def view_flights():
    conn = sqlite3.connect("airport.db")
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM flights")
    rows = cursor.fetchall()

    conn.close()

    print("\nFlights Table:")
    for row in rows:
        print(row)

In [14]:
def view_bookings():
    conn = sqlite3.connect("airport.db")
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM bookings")
    rows = cursor.fetchall()

    conn.close()

    print("\nBookings Table:")
    for row in rows:
        print(row)

In [15]:
view_users()
view_flights()
view_bookings()


Users Table:
('ali@gmail.com', 'Ali')
('ahmed@gmail.com', 'Ahmed')
('hassan@gmail.com', 'Hassan')
('ikrammir69@gmail.com', 'ikram')

Flights Table:
(1, 'Karachi', 'Lahore', '2026-02-20', '09:00 AM', 10)
(2, 'Karachi', 'Lahore', '2026-02-20', '03:00 PM', 15)
(3, 'Lahore', 'Karachi', '2026-02-21', '11:00 AM', 12)
(4, 'Islamabad', 'Karachi', '2026-02-22', '01:00 PM', 8)
(5, 'Karachi', 'Islamabad', '2026-02-23', '06:00 PM', 20)

Bookings Table:
(1, 'ali@gmail.com', 1, 'Booked')
(2, 'ahmed@gmail.com', 2, 'Booked')
(3, 'hassan@gmail.com', 3, 'Cancelled')


In [16]:
def booked_flight(status, id):
    conn = sqlite3.connect('airport.db')
    cursor = conn.cursor()
    cursor.execute("UPDATE flights SET status = ? WHERE id = ?", (status, id))
    conn.commit()

    cursor.execute("SELECT * FROM flights WHERE id = ?", (id,))
    row = cursor.fetchone()  # get the updated row

    conn.close()
    return row  # return the tuple

In [17]:
def cancel_ticket(status, id):
    conn = sqlite3.connect('airport.db')
    cursor = conn.cursor()
    cursor.execute("UPDATE flights SET status = ? WHERE id = ?", (status, id))
    conn.commit()

    cursor.execute("SELECT * FROM flights WHERE id = ?", (id,))
    row = cursor.fetchone()

    conn.close()
    return row

In [18]:
cancel_ticket('NOT Booked',101)

OperationalError: no such column: status

In [19]:
view_database()

NameError: name 'view_database' is not defined

In [20]:
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv('GEMINI_API_KEY')


In [21]:
client = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key = OPENAI_API_KEY)

In [22]:
response = client.chat.completions.create(model = "gemini-2.5-flash",messages={'role':'user','content':'hellow'})

In [23]:
print(response.choices[0].message.content)

Hello! How can I help you today?


1️⃣ Login / Register User

In [43]:
def login_user(email, name):
    conn = sqlite3.connect("airport.db")
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM user WHERE email = ?", (email,))
    user = cursor.fetchone()

    if user:
        conn.close()
        return json.dumps({
            "status": "exists",
            "message": "User already exists"
        })
    else:
        cursor.execute(
            "INSERT INTO user (email, name) VALUES (?, ?)",
            (email, name)
        )
        conn.commit()
        conn.close()
        return json.dumps({
            "status": "created",
            "message": "User created successfully"
        })        


In [44]:
"""
tool_login_user ={
    'name':'login_user',
    'description':'call this tool when the user went to login or check their login',
    'parameters': {
        'type':'object',
        'properties':{
            'email':{
            'type':'string',
            'description':'email is primary key in database required to create the new login or check if login already existed'
            },
            'name':{
                'type':'string',
                'description':'This is the name of user went to login'
            },        
             },
            'required':['email','name'],
            }
}
"""

"\ntool_login_user ={\n    'name':'login_user',\n    'description':'call this tool when the user went to login or check their login',\n    'parameters': {\n        'type':'object',\n        'properties':{\n            'email':{\n            'type':'string',\n            'description':'email is primary key in database required to create the new login or check if login already existed'\n            },\n            'name':{\n                'type':'string',\n                'description':'This is the name of user went to login'\n            },        \n             },\n            'required':['email','name'],\n            }\n}\n"

2️⃣ Search Flights

In [45]:
def search_flights(source, destination, date):
    conn = sqlite3.connect("airport.db")
    cursor = conn.cursor()

    cursor.execute("""
        SELECT * FROM flights
        WHERE source = ?
        AND destination = ?
        AND date = ?
        AND seats_available > 0
    """, (source, destination, date))

    flights = cursor.fetchall()
    conn.close()

    return flights

In [46]:
"""
tool_search_flights = {
    'name':'search_flights',
    'description':'Call this tool when user went to know about  flights information seats ,time etc',
    'parameters': {
        'type':'object',
        'properties':{
            'source':{
                'type':'string',
                'description':'This location or source of user from they went to travelled'
            },
            'destination':{
                'type':"string",
                'destination':'This is location or destination where user went to go'
            },
            'date':{
                'type':'string',
                'description':'This is date provided by the user'
            }
        },
        'required':['source','destination','date']
    }
}
"""

'\ntool_search_flights = {\n    \'name\':\'search_flights\',\n    \'description\':\'Call this tool when user went to know about  flights information seats ,time etc\',\n    \'parameters\': {\n        \'type\':\'object\',\n        \'properties\':{\n            \'source\':{\n                \'type\':\'string\',\n                \'description\':\'This location or source of user from they went to travelled\'\n            },\n            \'destination\':{\n                \'type\':"string",\n                \'destination\':\'This is location or destination where user went to go\'\n            },\n            \'date\':{\n                \'type\':\'string\',\n                \'description\':\'This is date provided by the user\'\n            }\n        },\n        \'required\':[\'source\',\'destination\',\'date\']\n    }\n}\n'

3️⃣ Book Flight

In [47]:
def book_flight(user_email, flight_id):
    conn = sqlite3.connect("airport.db")
    cursor = conn.cursor()

    # Check seat availability
    cursor.execute("SELECT seats_available FROM flights WHERE flight_id = ?", (flight_id,))
    result = cursor.fetchone()

    if not result:
        conn.close()
        return {"error": "Flight not found"}

    seats = result[0]

    if seats <= 0:
        conn.close()
        return {"error": "No seats available"}

    # Insert booking
    cursor.execute("""
        INSERT INTO bookings (user_email, flight_id, status)
        VALUES (?, ?, ?)
    """, (user_email, flight_id, "Booked"))

    # Reduce seat count
    cursor.execute("""
        UPDATE flights
        SET seats_available = seats_available - 1
        WHERE flight_id = ?
    """, (flight_id,))

    conn.commit()
    conn.close()

    return {"message": "Flight booked successfully"}

In [48]:
"""
tool_book_flight={
    'name':'book_flight',
    'description':'Call this tool when user went to book flight',
    'parameters': {
        'type':'object',
        'properties':{
            'user_email':{
                'type':'string',
                'description':'This is forign key used in booking table'
            },
            'flight_id':{
                'type':'string',
                'description':'This is flight id requireded '
            }
            

        },
        'required':['user_email','flight_id']
    }
}
"""

#user_email, flight_id

"\ntool_book_flight={\n    'name':'book_flight',\n    'description':'Call this tool when user went to book flight',\n    'parameters': {\n        'type':'object',\n        'properties':{\n            'user_email':{\n                'type':'string',\n                'description':'This is forign key used in booking table'\n            },\n            'flight_id':{\n                'type':'string',\n                'description':'This is flight id requireded '\n            }\n\n\n        },\n        'required':['user_email','flight_id']\n    }\n}\n"

Cancel Booking

In [49]:
def cancel_booking(booking_id):
    conn = sqlite3.connect("airport.db")
    cursor = conn.cursor()

    # Get flight_id from booking
    cursor.execute("SELECT flight_id FROM bookings WHERE booking_id = ?", (booking_id,))
    result = cursor.fetchone()

    if not result:
        conn.close()
        return {"error": "Booking not found"}

    flight_id = result[0]

    # Update booking status
    cursor.execute("""
        UPDATE bookings
        SET status = ?
        WHERE booking_id = ?
    """, ("Cancelled", booking_id))

    # Increase seat count
    cursor.execute("""
        UPDATE flights
        SET seats_available = seats_available + 1
        WHERE flight_id = ?
    """, (flight_id,))

    conn.commit()
    conn.close()

    return {"message": "Booking cancelled successfully"}

In [50]:
"""
cancel_booking_tool = {
        "name": "cancel_booking",
        "description": "Cancel a flight booking using booking ID",
        "parameters": {
            "type": "object",
            "properties": {
                "booking_id": {
                    "type": "integer",
                    "description": "The booking ID to cancel"
                }
            },
            "required": ["booking_id"]
        }
    }
"""    


'\ncancel_booking_tool = {\n        "name": "cancel_booking",\n        "description": "Cancel a flight booking using booking ID",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "booking_id": {\n                    "type": "integer",\n                    "description": "The booking ID to cancel"\n                }\n            },\n            "required": ["booking_id"]\n        }\n    }\n'

View User Bookings

In [51]:
def view_user_bookings(user_email):
    conn = sqlite3.connect("airport.db")
    cursor = conn.cursor()

    cursor.execute("""
        SELECT 
            bookings.booking_id,
            flights.source,
            flights.destination,
            flights.date,
            flights.time,
            bookings.status
        FROM bookings
        JOIN flights ON bookings.flight_id = flights.flight_id
        WHERE bookings.user_email = ?
    """, (user_email,))

    data = cursor.fetchall()
    conn.close()

    return data

In [52]:
"""
tool_view_user_booking ={
    'name':'view_user_bookings',
    'description':'call this tool when user went to  view all of their booking details',
    'parameters':{
        'properties':{
            'type':'object',
            'user_email':{
                'type':'string',
                'description':'this is key to acces the all information of the user '
            }
        },
        'required':['user_email']
    }
}
"""

"\ntool_view_user_booking ={\n    'name':'view_user_bookings',\n    'description':'call this tool when user went to  view all of their booking details',\n    'parameters':{\n        'properties':{\n            'type':'object',\n            'user_email':{\n                'type':'string',\n                'description':'this is key to acces the all information of the user '\n            }\n        },\n        'required':['user_email']\n    }\n}\n"

In [53]:
tool_login_user = {
    "type": "function",
    "function": {
        "name": "login_user",
        "description": "Call this tool when user wants to login or check login",
        "parameters": {
            "type": "object",
            "properties": {
                "email": {
                    "type": "string",
                    "description": "User email (primary key)"
                },
                "name": {
                    "type": "string",
                    "description": "Name of the user"
                }
            },
            "required": ["email", "name"]
        }
    }
}


tool_search_flights = {
    "type": "function",
    "function": {
        "name": "search_flights",
        "description": "Call when user wants flight information",
        "parameters": {
            "type": "object",
            "properties": {
                "source": {
                    "type": "string",
                    "description": "Departure location"
                },
                "destination": {
                    "type": "string",
                    "description": "Arrival location"
                },
                "date": {
                    "type": "string",
                    "description": "Travel date"
                }
            },
            "required": ["source", "destination", "date"]
        }
    }
}


tool_book_flight = {
    "type": "function",
    "function": {
        "name": "book_flight",
        "description": "Call when user wants to book a flight",
        "parameters": {
            "type": "object",
            "properties": {
                "user_email": {
                    "type": "string",
                    "description": "User email (foreign key)"
                },
                "flight_id": {
                    "type": "integer",
                    "description": "Flight ID to book"
                }
            },
            "required": ["user_email", "flight_id"]
        }
    }
}


cancel_booking_tool = {
    "type": "function",
    "function": {
        "name": "cancel_booking",
        "description": "Cancel booking using booking ID",
        "parameters": {
            "type": "object",
            "properties": {
                "booking_id": {
                    "type": "integer",
                    "description": "Booking ID to cancel"
                }
            },
            "required": ["booking_id"]
        }
    }
}


tool_view_user_booking = {
    "type": "function",
    "function": {
        "name": "view_user_bookings",
        "description": "View all bookings of a user",
        "parameters": {
            "type": "object",
            "properties": {
                "user_email": {
                    "type": "string",
                    "description": "User email to fetch bookings"
                }
            },
            "required": ["user_email"]
        }
    }
}

In [54]:
tools = [tool_login_user,tool_search_flights,tool_book_flight,cancel_booking_tool,tool_view_user_booking]

In [55]:
"""
tools = [
    {"type": "function", "function": tool_login_user},
    {"type": "function", "function": tool_search_flights},
    {"type": "function", "function": tool_book_flight},
    {"type": "function", "function": cancel_booking_tool},
    {"type": "function", "function": tool_view_user_booking}
]
"""

'\ntools = [\n    {"type": "function", "function": tool_login_user},\n    {"type": "function", "function": tool_search_flights},\n    {"type": "function", "function": tool_book_flight},\n    {"type": "function", "function": cancel_booking_tool},\n    {"type": "function", "function": tool_view_user_booking}\n]\n'

In [56]:
"""
import json
print("Tools structure:")
print(json.dumps(tools, indent=2))
"""

'\nimport json\nprint("Tools structure:")\nprint(json.dumps(tools, indent=2))\n'

In [57]:
system_message = """
You are an intelligent Airport Assistant.

You help users with:
1= the user to create the account
2= help in searching for the flight
3=help in booking the flight
4=help in cancel the booking flight
5=help in checking for their booking flight

Rules:
- If the user wants to login , call the tool "tool_login_user".
- If the user wants to search or find a flight, call the tool "tool_search_flight".
- If the user wants to cancel booked flight, call the tool "cancel_booking_tool".
- If the user wants to book the flights, call the tool "tool_book_flight".
- If the user wants to view all of their booked flights, call the tool "tool_book_flight".
- Do NOT manually generate booking confirmation without calling the correct tool.
- Always extract the required parameters from the message and pass it to the tool.
- Do not answer from memory. Always rely on the tool for database-related tasks.
"""

In [66]:
#function for tool handling
def handle_tool(content):
    content1 = content.tool_calls[0]
    response = {}

    if content1.function.name == 'login_user':
        args = json.loads(content1.function.arguments) if isinstance(content1.function.arguments, str) else content1.function.arguments
        arg_email = args['email']
        arg_name = args['name']
        result = login_user(arg_email,arg_name)         
        response = {
            'role': 'tool',
            'content': result,
            'tool_call_id': content1.id
        }

    elif content1.function.name == 'search_flights':

        args = json.loads(content1.function.arguments) if isinstance(content1.function.arguments, str) else content1.function.arguments
    
        arg_source = args['source']
        arg_destination = args['destination']
        arg_date = args['date']

        result = search_flights(arg_source, arg_destination, arg_date)

        if not result:
            result_str = "No flights found for the given route and date."
        else:
            flights = []
        for row in result:
            flights.append(
                f"Flight ID: {row[0]}, Source: {row[1]}, Destination: {row[2]}, Date: {row[3]}, Seats Available: {row[4]}"
            )
            result_str = "\n".join(flights)

        response = {
            "role": "tool",
         "content": result_str,
            "tool_call_id": content1.id
        }
    elif content1.function.name == 'book_flight':
        args = json.loads(content1.function.arguments) if isinstance(content1.function.arguments, str) else content1.function.arguments
        args_user_email = args['user_email']
        args_flight_id = args['flight_id']
        result = book_flight(args_user_email, args_flight_id)  
        response = {
            'role': 'tool',
            'content': result,
            'tool_call_id': content1.id
        }
    elif content1.function.name =='cancel_booking':
        args = json.loads(content1.function.arguments) if isinstance(content1.function.arguments,str) else content1.function.arguments  
        args_booking_id = args['booking_id']
        result = cancel_booking(args_booking_id)
        response = {
            'tool':'tool',
            'content':result,
            'tool_call_id': content1.id
        }  

    elif content1.function.name == 'view_user_bookings':
        args = json.loads(content1.function.arguments) if isinstance(content1.function.arguments,str) else content1.function.arguments
        args_user_email = args['user_email']
        result = view_user_bookings(args_user_email)
        response ={
            'tool':'tool',
            'content':result,
            'tool_call_id':content1.id
        }
    return response

In [59]:
# Chat function to interact with Gemini LLM
def chat_with_ai(message, history):
    # Prepare message history
    history = [{'role': h['role'], 'content': h['content']} for h in history]
    messages = [{'role': 'system', 'content': system_message}] + history + [{'role': 'user', 'content': message}]

    # First call to LLM
    first_response = client.chat.completions.create(
        model='gemini-2.5-flash',
        messages=messages,
        tools=tools,
        tool_choice='auto'
    )

    # If tool call happens
    if first_response.choices[0].finish_reason == 'tool_calls':
        content = first_response.choices[0].message
        tool_call = content.tool_calls[0]

        # Handle tool call
        result = handle_tool(content)

        # Append tool call and result back to messages
        messages.append({
            "role": "assistant",
            "tool_calls": content.tool_calls
        })
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": result['content']
        })

        # Second call to LLM after tool result
        second_response = client.chat.completions.create(
            model='gemini-2.5-flash',
            messages=messages,
            tools=tools,
            tool_choice='auto'
        )
        return second_response.choices[0].message.content

    # If no tool call, just return normal LLM reply
    return first_response.choices[0].message.content

In [60]:
import gradio as gr

In [67]:
gr.ChatInterface(fn=chat_with_ai,type='messages').launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
